#### [Agents SDK Course](https://www.aurelio.ai/course/agents-sdk)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/agents-sdk-course/blob/main/chapters/09-rag-agent.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/agents-sdk-course/blob/main/chapters/09-rag-agent.ipynb)

## RAG Agent

**R**etrieval **A**ugmented **G**eneration (RAG) is a powerful technique that enables AI agents to access and leverage external knowledge sources beyond their training data. In this tutorial, we'll build a RAG agent that can answer questions about the JFK assassination files using OpenAI's Agents SDK and Pinecone vector database.

RAG is particularly useful when:
- You need up-to-date information beyond the model's training cutoff
- You have domain-specific documents or proprietary data
- You want to reduce hallucinations by grounding responses in factual sources
- You need to cite sources for transparency and verification

By the end of this tutorial, you'll have built an agent that can search through historical documents and provide accurate, sourced answers about the JFK files.


### Prerequisites

Before we begin, let's install the required packages:

In [ ]:
!pip install -qU \
    "openai-agents==0.1.0" \
    "pinecone==7.0.2" \
    "datasets==3.6.0" \
    "semantic-chunkers==0.1.1"

We also need API keys for OpenAI and Pinecone. You can get:
- An OpenAI API key from the [OpenAI Platform](https://platform.openai.com/api-keys)
- A Pinecone API key from the [Pinecone Console](https://app.pinecone.io/)

In [1]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OPENAI_API_KEY: "
)

### Testing LLM Knowledge Limitations

Before implementing RAG, let's first demonstrate why it's needed. We'll create a basic agent and test its knowledge about specific topics to show the limitations of relying solely on the model's training data.

In [2]:
from agents import Agent

agent = Agent(
    name="Agent",
    model="gpt-4.1-mini"
)

We'll ask our agent `"where was Oswald in october 1959?"`:

In [3]:
from agents import Runner

query = "where was Lee Harvey Oswald in october 1959?"

result = await Runner.run(
    starting_agent=agent,
    input=query,
)

print(result.final_output)

In October 1959, Lee Harvey Oswald was in the United States, specifically in the state of Texas. During this period, Oswald was living with his family in Fort Worth, Texas. This was a few years before his defection to the Soviet Union in 1959, which happened later that year in October when he traveled to the Soviet Union.


Oswald was _also_ in Helsinki, Finland in October 1959 according to the [JFK files](https://www.archives.gov/files/research/jfk/releases/2025/0318/104-10004-10156.pdf) - which our agent missed. We can try and tease out this information:

In [4]:
result = await Runner.run(
    starting_agent=agent,
    input=[
        {"role": "user", "content": query},
        {"role": "assistant", "content": result.final_output},
        {"role": "user", "content": "did he go anywhere else?"}
    ],
)

print(result.final_output)

Yes, Lee Harvey Oswald did travel elsewhere in October 1959. More specifically, in early October 1959, Oswald traveled from the United States to the Soviet Union. On October 16, 1959, he sailed from New Orleans to Le Havre, France, then traveled to the Soviet Union, where he eventually settled in Minsk, Belarus, which was part of the USSR at the time.

So, to clarify:

- Before mid-October 1959, Oswald was in the United States (Texas and elsewhere).
- On October 16, 1959, he left the U.S., traveling via France to the Soviet Union.
- After arrival, he stayed in the Soviet Union for about two and a half years until he returned to the U.S. in mid-1962.

If you want detailed information about any specific dates or places during that period, feel free to ask!


Our agent is clearly not aware of Oswald's trip to Helsinki - that is because the underlying LLM has not seen that information during it's training process. We call information learned during LLM training **parametric knowledge**, ie knowledge stored within the model _parameters_.

LLMs can also make use of **source knowledge** to answer questions. Source knowledge refers to information provided to an LLM via a prompt, either provided via the user, the LLM instructions, or in our case - via an external database - ie with **R**etrieval **A**ugmented **G**eneration (RAG). Before we build out our RAG pipeline, let's see if our LLM can answer our question when we provide the relevant information about Oswald's whereabouts via our `instructions`.

In [5]:
source_knowledge = (
    "~SECRET~\n"
    "1 June 1964\n"
    "\n"
    "## MEMO FOR THE RECORD\n"
    "\n"
    "1. At 0900 this morning I talked with Frank Friberg recently "
    "returned COS Helsinki re Warren Commission inquiry concerning "
    "the timetable of Oswald's stay in Finland in October 1959, including "
    "his contact with the Soviet Consulate there. (Copy of the Commission "
    "letter of 25 May 64 and State Cable of 22 May 64 attached.)"
)

agent = Agent(
    name="Agent",
    instructions=(
        "You are an assistant specialized in answering questions about the JFK assassination"
        "and related documents.\n"
        "Here is some additional context that you can use:\n"
        f"{source_knowledge}\n"
    ),
    model="gpt-4.1-mini"
)

Let's ask our original `query` again:

In [6]:
result = await Runner.run(
    starting_agent=agent,
    input=query,
)

print(result.final_output)

In October 1959, Lee Harvey Oswald was in Finland. This is evidenced by the memo referencing a conversation with Frank Friberg, who had recently returned as COS (Chief of Station) in Helsinki, regarding Oswald's timetable and contacts with the Soviet Consulate in Finland during that month.


Perfect, this is much better! Now what we just did works for this simple example, but it doesn't scale. If we want an agent that can answer any question and use context from _all_ of the JFK files, we need to build a RAG pipeline.

## Building a RAG Pipeline

A RAG pipeline actually requires two _core_ pipelines - an **ingestion pipeline** and a **retrieval pipeline**. At a high level those pipelines are responsible for:

* **Ingestion** handles the initial data preparation, embedding, and indexing. We'll explain those steps in more detail soon, but the tldr is that the ingestion pipeline will transform a set of unstructured and messy PDFs into a "second brain" for our agent, ie the _source knowledge_.

* **Retrieval** handles the query-time retrieval of information. It defines how we access and retrieve source knowledge from our second brain.

Naturally, we need to first develop our **ingestion pipeline** so that we can populate our second brain before we use the **retrieval pipeline** to retrieve anything.

### Ingestion Pipeline

The ingestion pipeline consists of three (or four) steps:

0. **Process the PDF** into plain text - with the `aurelio-ai/jfk-files` dataset (below) this step has been completed.

1. **Chunk** the plain text into smaller segments (a good rule of thumb is ~300-400 tokens per chunk).

2. **Embed** each chunk with OpenAI's `text-embedding-3-small` to create _vectors_.

3. **Index** those vectors in Pinecone with metadata like _source URL_, _document title_, etc.

![JFK document ingestion pipeline, covering PDF text to chunked text, embedding those chunks into semantically meaningful vector embeddings, and sending those vector embeddings to a vector database](../assets/jfk-ingestion-pipeline.png)

To begin, we'll start at step **0** and download the pre-parsed JFK files.


### Loading the JFK Files Dataset

We'll use a dataset of the JFK files, which we will pull from the [Hugging Face Hub](). This dataset contains historical documents that our agent will search through to answer questions:

In [7]:
from datasets import load_dataset

dataset = load_dataset(
    "aurelio-ai/jfk-files",
    split="train"
)

/Users/jamesbriggs/Documents/aurelio/agents-sdk-course/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's examine a sample document to understand the data structure:

In [8]:
dataset[1]

{'id': 'doc_8c4732b4_28af_4ee8_8be0_e10b6f19ee3a',
 'filename': '104-10105-10126.pdf',
 'url': 'https://www.archives.gov/files/research/jfk/releases/2025/0318/104-10105-10126.pdf',
 'date': datetime.datetime(2025, 3, 18, 0, 0),
 'content': '[104-10105-10126\n\n<!-- image -->\n\n3 June 1976\n\nMEMORANDUM FOR:\n\nChief External Activities Branch Office of Security\n\nFROM\n\nJohn H . Stein Deputy Chief Soviet/East European\n\nDivision\n\nSUBJECT\n\nManuscript by David Phillips.\n\n\n- 1\\_ At the risk of tilting continually at windmills it is the undersigned\' s view and that of innumerable colleagues that Mr \\_ Phillips book should not be published In the first instance\\_ those portions we have read are superficial and give the reader the impression that the profession Mr practiced s0 well is one of derring frivolity and foolishness \\_ More seriously , the entire book is based on knowledge acquired by Mr Phillips during his career in the Agency Whether a given sentence is in the publ

Each document contains:
- `id`: Unique identifier for the document
- `filename`: Name of the PDF file
- `url`: Link to the original document
- `date`: Publication date
- `content`: The full text content
- `pages`: Number of pages in the original document

### Chunking our Data

Step **1** in our ingestion pipeline is to _chunk_ our dataset. As mentioned we will be splitting each PDF into chunks of ~400 tokens. We'll also handle cases where a PDF contains little-to-no information by not indexing that PDF, and cases where our final chunk is too small to be relevant by appending it to the previous chunk.

We use the lightweight [`semantic-chunkers`](https://github.com/aurelio-labs/semantic-chunkers) library and a simple `RegexChunker` for chunking. We will set the token limit for each chunk to `400` tokens:

In [9]:
from semantic_chunkers import RegexChunker

chunker = RegexChunker(max_chunk_tokens=400)

2025-06-10 15:07:56 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"


We chunk a doc like so:

In [10]:
chunks = chunker(docs=[dataset[1]['content']])
chunks

[[Chunk(splits=['[104-10105-10126', '<!-- image -->', '3 June 1976', 'MEMORANDUM FOR:', 'Chief External Activities Branch Office of Security', 'FROM', 'John H .', 'Stein Deputy Chief Soviet/East European', 'Division', 'SUBJECT', 'Manuscript by David Phillips.', "- 1\\_ At the risk of tilting continually at windmills it is the undersigned' s view and that of innumerable colleagues that Mr \\_ Phillips book should not be published In the first instance\\_ those portions we have read are superficial and give the reader the impression that the profession Mr practiced s0 well is one of derring frivolity and foolishness \\_ More seriously , the entire book is based on knowledge acquired by Mr Phillips during his career in the Agency Whether a given sentence is in the public domain or not Mr \\_ Phillips should not put his stamp of authenticity on that sentence If publish Mr Phillips must, and if the \\_ Agency cannot legally him, then be it\\_ Hopefully while there is still time , someone wi

This outputs a list of a list of `Chunk` objects. These `Chunk` objects contain many smaller `splits`, which can be thought of as chunks within chunks. We can view the chunks in a cleaner way using `chunker.print` on a `list[Chunk]` object like so:

In [11]:
chunker.print(chunks[0])

Split 1, tokens 367, triggered by: token limit
[104-10105-10126 <!-- image --> 3 June 1976 MEMORANDUM FOR: Chief External Activities Branch Office of Security FROM John H . Stein Deputy Chief Soviet/East European Division SUBJECT Manuscript by David Phillips. - 1\_ At the risk of tilting continually at windmills it is the undersigned' s view and that of innumerable colleagues that Mr \_ Phillips book should not be published In the first instance\_ those portions we have read are superficial and give the reader the impression that the profession Mr practiced s0 well is one of derring frivolity and foolishness \_ More seriously , the entire book is based on knowledge acquired by Mr Phillips during his career in the Agency Whether a given sentence is in the public domain or not Mr \_ Phillips should not put his stamp of authenticity on that sentence If publish Mr Phillips must, and if the \_ Agency cannot legally him, then be it\_ Hopefully while there is still time , someone will push fo

We'll need the text content from our chunks which we access via the `content` attribute:

In [12]:
chunks[0][0].content

"[104-10105-10126 <!-- image --> 3 June 1976 MEMORANDUM FOR: Chief External Activities Branch Office of Security FROM John H . Stein Deputy Chief Soviet/East European Division SUBJECT Manuscript by David Phillips. - 1\\_ At the risk of tilting continually at windmills it is the undersigned' s view and that of innumerable colleagues that Mr \\_ Phillips book should not be published In the first instance\\_ those portions we have read are superficial and give the reader the impression that the profession Mr practiced s0 well is one of derring frivolity and foolishness \\_ More seriously , the entire book is based on knowledge acquired by Mr Phillips during his career in the Agency Whether a given sentence is in the public domain or not Mr \\_ Phillips should not put his stamp of authenticity on that sentence If publish Mr Phillips must, and if the \\_ Agency cannot legally him, then be it\\_ Hopefully while there is still time , someone will push for appropriate legislation to this fooli

In the next step we'll setup our Pinecone vector DB and begin **embedding _and_ indexing** our data in one step - while indexing we'll be performing the above chunking logic across all our docs before they're embedded.

### Embedding and Indexing


To enable semantic search over our documents, we'll use [Pinecone](https://www.pinecone.io/) - a fully managed vector database. Vector databases allow us to store and search through vector embeddings (numerical representations of text) to find semantically similar content. There are many vector DB options out there, alongside Pinecone we also recommend [Qdrant](https://qdrant.tech/) and [pgvector](https://github.com/pgvector/pgvector).

First, let's set up our Pinecone API key which we can find in the [Pinecone console](https://app.pinecone.io/):

In [13]:
from pinecone import Pinecone

os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY") or getpass(
    "Enter PINECONE_API_KEY: "
)

# initializing the pinecone client
pc = Pinecone()

Now we'll create a Pinecone index to store our vector embeddings. We specify the following:

- We want to use AWS via `cloud=CloudProvider.AWS` in Pinecone's free tier region via `region=AwsRegion.US_EAST_1`.
- We use the `llama-text-embed-v2` embedding model hosted by Pinecone - by default the index will be configured for this model.
- We specify that the text content that should be embedding by our model will be provided to Pinecone via the `content` metadata field.

In [14]:
from pinecone import AwsRegion, CloudProvider

# set our index name, you can change this to whatever you like
index_name = "agents-sdk-course-jfk-files"

# if the index doesn't exist, create it
if index_name not in pc.list_indexes().names():
    pc.create_index_for_model(
        name=index_name,
        cloud=CloudProvider.AWS,
        region=AwsRegion.US_EAST_1,
        embed={
            "model": "llama-text-embed-v2",
            "field_map": {
                "text": "content"
            }
        }
    )

index = pc.Index(index_name)

Let's check if our index is empty (it should be on first run):

In [15]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

To embed and index a chunk, we can do the following:

In [16]:
doc = dataset[1]

# chunk the doc
chunks = chunker(docs=[doc['content']])

# create a list of dictionary records
records = [
    {
        "id": doc['id']+f"-{i}",
        "content": chunk.content,
        "filename": doc['filename'],
        "url": doc['url'],
        "date": doc['date'].isoformat(),
        "pages": doc['pages']
    } for i, chunk in enumerate(chunks[0])
]

# embed and index
index.upsert_records(
    namespace="default",
    records=records
)

Now we should see that our index contains three records inside the `default` namespace:

In [17]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

Perfect! Now we simply repeat that process for all of our docs. We will do this in batches to avoid excessive network calls with small packages.

In [18]:
from tqdm.auto import tqdm

records = []
for doc in tqdm(dataset):
    # perform a quick length check of our docs to avoid excessively small docs
    if len(doc['content']) < 100:
        # nothing less than 100 chars
        continue
    # chunk the docs
    chunks = chunker(docs=[doc['content']])
    for i, chunk in enumerate(chunks[0]):
        records.append(
            {
                "id": doc['id']+f"-{i}",
                "content": chunk.content,
                "filename": doc['filename'],
                "url": doc['url'],
                "date": doc['date'].isoformat(),
                "pages": doc['pages']
            }
        )
    if len(records) >= 64:
        # if we have a particularly long doc, we'll need to split up the batch
        for i in range(0, len(records), 96):
            # 96 is the max number of records we can upsert in one go
            batch = records[i:i+96]
            # embed and index the batch
            index.upsert_records(
                namespace="default",
                records=batch
            )
        records = []

index.describe_index_stats()

100%|██████████| 606/606 [00:22<00:00, 26.51it/s]


{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'default': {'vector_count': 1804}},
 'total_vector_count': 1804,
 'vector_type': 'dense'}

That's our **ingestion pipeline** complete and we're ready to move on to the **retrieval pipeline**.

## Retrieval Pipeline

Our retrieval pipeline is what will be used to retrieve the right source knowledge for our agent at query-time. We will be implementing this via an Agent SDK `@function_tool` but before we do so let's directly test retrieval.

As we're using Pinecone's integrated inference (ie both indexing and embedding are handled by Pinecone) the retrieval pipeline is incredibly simple.

In [19]:
results = index.search(
    namespace="default",
    query={
        "inputs": {"text": query},
        "top_k": 5
    },
    fields=["content", "url", "pages"]
)

results

{'result': {'hits': [{'_id': 'doc_5f3f1ea3_aced_45e6_b35d_70bfe503daf6-1',
                      '_score': 0.36479848623275757,
                      'fields': {'content': '- At 0900 this morn I talked with '
                                            'Frank Friberg recently recurned '
                                            'COS Helsinki re Warren '
                                            'Commlission inqulry concerni the '
                                            'tímetable of Oswald stay in '
                                            'Finland in October 1959 ng his '
                                            'contact with the Soviet Consulate '
                                            'there. including letler of 25 64 '
                                            'and State Cable (Copy of the '
                                            'Comnission of 22 64 attached.) '
                                            'ing May May - 2 Friberg gave the '
              

Let's format these a little nicer:

In [ ]:
from IPython.display import Markdown, display

# let's print out the results
results_str = """
| Score | Content | Pages | URL |
|-------|---------|-------|-----|
"""
for result in results["result"]["hits"]:
    results_str += (
        f"| {result['_score']:.2f} "
        "| " + result['fields']['content'].replace('|', '\|') + " "
        f"| {result['fields']['pages']} "
        f"| {result['fields']['url']} |" "\n"
    )

display(Markdown(results_str))

<>:11: SyntaxWarning: invalid escape sequence '\|'
<>:11: SyntaxWarning: invalid escape sequence '\|'
/var/folders/0d/qsc6nk1x38gbh2qd8s0mmq740000gn/T/ipykernel_19269/2697269785.py:11: SyntaxWarning: invalid escape sequence '\|'
  f"| {result['fields']['content'].replace('|', '\|')} "



| Score | Content | Pages | URL |
|-------|---------|-------|-----|
| 0.36 | - At 0900 this morn I talked with Frank Friberg recently recurned COS Helsinki re Warren Commlission inqulry concerni the tímetable of Oswald stay in Finland in October 1959 ng his contact with the Soviet Consulate there. including letler of 25 64 and State Cable (Copy of the Comnission of 22 64 attached.) ing May May - 2 Friberg gave the folloving Information= - Ic takes 25 minutes to drive the airport to downtown Helsinki; from - b By taxi, it would take no more than 5 minutes to reach the Soviet consulate; - c The Soviet consulate probably closed at 1300 hvurs Jocal Saturdays in 1959; - Passenger lists (manifests) at che Consulute in Helsinki are retained for six months only and chen are destroyed Mr Robert Fulton (CIA) was consular official there at the time - A copy of State 5 cable inquiry would go to th( Helsinki Station and would assist in preparation of reply they - 3 Mr\_ Friberg agreed that it would be worchvhile É0 cable che Station concerning points not covered by State in their inquiry He suggested changes incorporated into the cable sent Helsinki to <!-- image --> <!-- image --> Document Number for FOIR Roview on As DOC 340 RECORD COPY Docld:32106269 Page 3 <!-- image --> | 3.0 | https://www.archives.gov/files/research/jfk/releases/2025/0318/104-10004-10143 (C06932208).pdf |
| 0.34 | 2025 RELEASE UNDER THE PRESIDENT JOHN F KENNEDY ASSASSINATION RECORDS ACT OF 1992 1002 Us/82/71 Dear Anthony Reqarding your letter 6/705 of 10 1982 , our records indicate that Valeriy Vladimirovich\_ Kostikoy traveled to Mexico France , Spain, the and Cuba during the period 1959-61In 1961 , he was assigned permanently to Mexico City as a consular officer and served there until August 1965. He was variously described as a translator vice-consul and attache During this tour he attempted to cultivate a Government employee assigned to our embassy in Mexico May \_ city. 20 \| - In September /October 1963 2 Lee Harvey Oswald approached the Soviet Embassy in Mexico City in an attempt to a visa allowing him to return to the USSR . Kostikov, as a consular officer, handled this visa request We have no infornation which indicates any relationship between these individuals other than for the purpose of Oswald's making his visa request. get Kostikov returned to Mexico City for à second tour of duty in July 1968During this tour he was again assigned to the consular section and was à second secretary It appeared that he was tasked with followz the activities of the Central American communist partiesánd left-wing groups and he met often with members Of these groups reportedly providing them with funds and technical guidanceIn July/August 1969 \_ Kostikov made an unusual TDY trip to Moscow lasting three weeks. (His family remained in Mexico. ) In July 1970 he made à four-day trip to Havana. ing | 2.0 | https://www.archives.gov/files/research/jfk/releases/2025/0318/104-10012-10022.pdf |
| 0.30 | ## MSMORAMDUM FOR: SUBJECI Audio Qpezation 1 In May 1959, Buildirg which is occupied ments. Bonsal and Miss Carolyn O. Stacey In Novenbez 1959, recorder 1959 2, By June: 1960, case weat to He mazaged to leas2 22 NCNA officeand arrazged íor 3 In August 1950, in Havaza 02 separate SR Division operation agreed to irstall the A secord FE Division case officer, Robert Meet, and he, together With the techniciars, installed &amp; pröbe microphone and tape 2partmeat, in the wall and plastered over . S-E-C-R-E-T precaution t0 provide a Bafe 4, As additional security Chief of Station, key\_ baven, and after consultation with the Act:ng Mrs. Marjorie to an oòtained. Mrs. Lannox was aeeded foz photographic apartment 5; The Station wräs irdefinitely the lease to holding the leases; It was; contracts 'aid recalling in order to permit\_continued man Christ (alias Carswell) ; Denbrunt) documented Taransky) and Thoratoa J. Chinese Natioralist aborted Chinese Cormunits. ment over the NCNA office. 6 other on scheduled apzoiatment Laanox was ertered into and September, At Neet, the the Cuban authorities and detained: 'picked up bY get 7 and is still in Havana, Neet euthorities. His wife bag been released suosequently returced: debriefed: Mrs. Vashington where she subaequertly isbeing debriefed, Lenaox jciatly interzogated Chzist was to have been Ialtreated. | 7.0 | https://www.archives.gov/files/research/jfk/releases/2025/0318/104-10095-10276.pdf |
| 0.30 | - 8 Uncropped copies of photographs of the "Mexico City Oswald" These include CIA photographs ## C/CI Memo to OLC (CI 622-77 29 Nov 77 : Eleven of the twelve photographs were released under the FOIAA copy of each of these photographs is included in the attached notebook - c (attached) A black notebook containing copies of 12 photographs of the "unidentified individual" who was seen entering or leaving the Cuban and Soviet Embassies in Mexico City. \| 1 ESCA_Request,9_Novenber_1977 (OLC #77-4894) for Inforuatlon generated by Or 1n the pogbession Of                                                                                                    \| \|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------\| \| 2 C/CI Memo to OLC 6 December 1977 (CI 632-77 by Holmes) re: Robert numerous re ferences to McKeown in FBI documents- The_identificationof FBI documents will_be forwarded by separate memorandum Ray \| 14-00000 \| HSCA Request, 22 December 1977 (OLC #77-5685/4) for access to files or documents on or referring to: 9 Priscilla Johnson McMillan Author of "Marina and Lee" interviewed Oswald in                                                                          \| \|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------\| \| DCD_112-78, 31 Jan 78: DCD response to_her request (P 76-1861) under the Privacy Act  Full text and sanitized copies of all this material shouldbe available from IPS . Also attached are four intelligence information reports which were not furnished in \| \| response to the PA request _ 23 7e6 PS                                                                                                                                                                                                                      \| 14-00000 MC WILLIE , Lewis J \|    \| by the Of CIA                                                                                                  \| \|----\|----------------------------------------------------------------------------------------------------------------\| | 78.0 | https://www.archives.gov/files/research/jfk/releases/2025/0318/104-10079-10016.pdf |
| 0.28 | 2025 RELEASE UNDER THE PRESIDENT JOHN F\_ KENNEDY ASSASSINATION RECORDS ACT OF 1992 <!-- image --> ## 1 June 1964 ## MEMO FOR IHE\_RECORD - At 0900 this morning I talked with Frank Friberg recently returned COS Helsínki re Warren Comm1ssion Inquiry concerning the timetable of Oswald s stay in Fínland October 1959, Including contact with the Soviet Consulate there. (Copy of the Commission letter of 25 May 64 and State Cable of 64 attached .) May - 2 . Friberg gave me the following information: - a, It takes 25 minutes to drive from the alrport to downtown Helsinki; - By taxi, it would take no more than 5 mInutes to reach the Soviet consulate; - The Sovlet consulate probably closed at 1300 hours local time 6n Saturdays in 1959; - d Passenger lists (manifests) at the U.S . Consulate in Helsinki are retäined for s1x months and then are destroyed . Mr \_ Robert Fulton (CIA) was U.S. consular officíal there at the time\_ only - e A copy of s cable inquiry would g0 Eo the Helsinki Station and they would assist in preparatlon of a reply. - 3. Mr \_ Friberg agreed that it would be worthwhile to cable the Station concerning points not covered by State In their Inquiry . He suggested changes incorporated into the cable sent to 780-340 <!-- image --> Document Number saME As 716-838 <!-- image --> Se0 Sanltized File Number 235 For | 3.0 | https://www.archives.gov/files/research/jfk/releases/2025/0318/104-10004-10156.pdf |


Now let's create a tool that our agent can use to search through the JFK documents. We use the `@function_tool` decorator to wrap the logic above and make the retrieval pipeline available to our agents.

In [ ]:
from agents import function_tool

@function_tool
def jfk_files_search(query: str) -> str:
    """This tool gives you search access to the full JFK files. To use this tool you
    should provide search queries with as much context as possible, and using natural
    language to describe the query.

    This tool will return five of the most relevant document chunks for your query,
    including the result's similarity score, the text content, the source page number,
    and source URL.
    """
    results = index.search(
        namespace="default",
        query={
            "inputs": {"text": query},
            "top_k": 5
        },
        fields=["content", "url", "pages"]
    )
    # format the results into a markdown string - this isn't essential for our LLM but
    # it helps
    source_knowledge = """
    | Score | Content | Pages | URL |
    |-------|---------|-------|-----|
    """
    for result in results["result"]["hits"]:
        source_knowledge += (
            f"| {result['_score']:.2f} "
            "| " + result['fields']['content'].replace('|', '\|') + " "
            f"| {result['fields']['pages']} "
            f"| {result['fields']['url']} |" "\n"
        )
    return source_knowledge

<>:30: SyntaxWarning: invalid escape sequence '\|'
<>:30: SyntaxWarning: invalid escape sequence '\|'
/var/folders/0d/qsc6nk1x38gbh2qd8s0mmq740000gn/T/ipykernel_19269/1644549257.py:30: SyntaxWarning: invalid escape sequence '\|'
  f"| {result['fields']['content'].replace('|', '\|')} "


Now we provide our `jfk_files_search` tool to an agent.

In [22]:
agent = Agent(
    name="JFK Document Assistant",
    model="gpt-4.1-mini",
    instructions=(
        "You are an assistant specialized in answering questions about the JFK "
        "assassination and related documents. When users ask questions about JFK, "
        "the assassination, or related historical events. Please write your answers "
        "in markdown and provide sources to support your answers."
    ),
    tools=[jfk_files_search]
)

## Building the Final RAG Agent

Now we can use our agent to discover who really assassinated JFK. First, let's confirm our agent is functional with our original query about Oswald's whereabouts in October 1959.

In [23]:
query

'where was Lee Harvey Oswald in october 1959?'

In [24]:
result = await Runner.run(
    starting_agent=agent,
    input=query,
)

display(Markdown(result.final_output))

2025-06-10 15:08:36 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-06-10 15:08:42 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


2025-06-10 15:08:37 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
2025-06-10 15:08:48 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
2025-06-10 15:08:58 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
2025-06-10 15:09:39 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
2025-06-10 15:09:50 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"


In October 1959, Lee Harvey Oswald was in Helsinki, Finland. This is supported by archival records indicating that in October 1959, Oswald made contacts with the Soviet Consulate in Helsinki. There are details about the timetable of Oswald's stay in Finland and his interactions with the Soviet Consulate during that time. 

The records also mention that passenger lists at the Soviet Consulate in Helsinki were retained only for six months and then destroyed, which might limit the extent of specific travel documentation. However, it is clear from the available reports that Lee Harvey Oswald was indeed in Helsinki in October 1959.

Source:  
- https://www.archives.gov/files/research/jfk/releases/2025/0318/104-10004-10143 (page 3)

If you want more detailed documentation or information about his activities there, I can help search further.

To keep things conversational we'll append our own queries and the agent responses to a `messages` list.

In [25]:
messages = [
    {"role": "user", "content": query},
    {"role": "assistant", "content": result.final_output}
]

In [26]:
messages.append(
    {"role": "user", "content": (
        "do the JFK files contain any information about doubts on Lee Harvey Oswald's "
        "involvement in the assassination?"
    )}
)

result = await Runner.run(
    starting_agent=agent,
    input=messages,
)

display(Markdown(result.final_output))

2025-06-10 15:08:44 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-06-10 15:08:54 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


The JFK files do contain information referencing doubts and alternative perspectives regarding Lee Harvey Oswald's involvement in the assassination of President John F. Kennedy. Some of the documents highlight allegations and claims suggesting Oswald may have been a Soviet intelligence agent or had connections with intelligence activities. For example:

- A handwritten autobiographical manuscript by an inmate claimed to be a former Cuban intelligence officer alleged that Oswald was a Soviet intelligence agent.
- There are memos and evaluations of intelligence files discussing these claims and making assessments of Oswald's possible connections.
- Some documents refer to investigations and reviews of Oswald's activities in Mexico and his interactions with the Soviet Embassy.
- These materials reflect uncertainties, different viewpoints, and investigative leads questioning the official narrative of Oswald as the lone assassin.

While there is not consensus, the JFK files demonstrate that doubts and inquiries into Oswald's exact role and possible wider conspiracies were present in government archives and investigations.

Sources:  
- [Document 104-10079-10016.pdf, page 2](https://www.archives.gov/files/research/jfk/releases/2025/0318/104-10079-10016.pdf): Contains memos about claims Oswald was a Soviet agent and investigations thereof.  
- Related JFK files discuss Oswald's activities in Mexico and intelligence evaluations:  
  https://www.archives.gov/files/research/jfk/releases/2025/0318/104-10012-10022.pdf  
  https://www.archives.gov/files/research/jfk/releases/2025/0318/104-10105-10120.pdf

If you want, I can provide excerpts or further detailed descriptions from these documents.

In [27]:
messages.extend(
    [
        {"role": "assistant", "content": result.final_output},
        {"role": "user", "content": "I see mentions of Oswald in Mexico, what did he do there?"}
    ]
)

result = await Runner.run(
    starting_agent=agent,
    input=messages,
)

display(Markdown(result.final_output))

2025-06-10 15:09:36 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


Lee Harvey Oswald's activities in Mexico, particularly in Mexico City, in late September and early October 1963, are documented in the JFK files and have been subject of significant investigation. Here's what is known based on the released files:

### What Oswald did in Mexico:

1. **Visits to Cuban and Soviet Embassies:**  
   Oswald visited both the Cuban and Soviet Embassies in Mexico City at least twice, on September 26 and 27, 1963. He attempted to obtain visas or some form of travel permission to Cuba and possibly the Soviet Union. These visits are documented through embassy records and surveillance.

2. **Seeking Travel Documents:**  
   Oswald sought a visa to travel to Cuba and reportedly wanted to go to the Soviet Union afterward. However, the Cuban Embassy denied his visa request.

3. **Surveillance by Mexican Authorities and CIA:**  
   Mexican police and possibly CIA agents monitored Oswald during his stay. His activities, including interactions and movements, were documented, with some photos and surveillance reports available in the files.

4. **Intent and Motive Ambiguity:**  
   The purpose behind Oswald's trip and embassy visits is interpreted in various ways. Some documents suggest he was trying to defect or establish connections; others propose he was acting in a more complex intelligence context, though no conclusive evidence confirms involvement beyond these trips.

### Importance of Mexico City Visits:  
The visits are significant because they occurred just weeks before the assassination of President Kennedy on November 22, 1963. The government's investigations examined whether these contacts linked Oswald to intelligence operations or conspiratorial plans.

### Sources:  
- [Mexican Government Police Report on Oswald's visit — 104-10105-10120](https://www.archives.gov/files/research/jfk/releases/2025/0318/104-10105-10120.pdf)  
- [CIA files on Oswald’s Mexico City trip — 104-10012-10022](https://www.archives.gov/files/research/jfk/releases/2025/0318/104-10012-10022.pdf)  
- Analysis in JFK files discusses at length these embassy visits and visa requests.

Let me know if you want a detailed timeline of Oswald’s movements in Mexico or excerpts from these documents!

In [28]:
messages.extend(
    [
        {"role": "assistant", "content": result.final_output},
        {"role": "user", "content": "Tell me more about Valeriy, is he relevant?"}
    ]
)

result = await Runner.run(
    starting_agent=agent,
    input=messages,
)

display(Markdown(result.final_output))

2025-06-10 15:09:38 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-06-10 15:09:49 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


Valeriy, specifically Valeriy Vladimirovich Kostikov, is mentioned in the JFK files as a Soviet consular officer posted to Mexico City during the time Lee Harvey Oswald was there in 1963. Here are key points about Kostikov and his relevance:

- Kostikov was a consular officer, translator, vice-consul, and attache in Mexico City between 1961 and 1965.
- In September/October 1963, Lee Harvey Oswald approached the Soviet Embassy in Mexico City to request a visa allowing him to return to the USSR. Kostikov handled this visa request.
- There is no information in the files indicating any personal relationship between Kostikov and Oswald beyond the visa request.
- After his Mexico City assignment, Kostikov was involved in monitoring Central American communist activities and reportedly provided funds and technical guidance to communist and left-wing groups.
- He made trips to Moscow and Havana during his assignments.
- There are references suggesting Kostikov was linked to the KGB, and discussions in the files consider whether he had any "Active Measures" (intelligence influence operations) role.
- His activities and possible connections are part of the broader FBI and CIA interest in Soviet intelligence activities in Mexico during the period Oswald was there.

In summary, Valeriy Kostikov was a Soviet consular official in Mexico at the time of Oswald’s visa request. While he is relevant as the Soviet Embassy officer who processed Oswald’s request, there is no definitive information in the JFK files proving a direct relationship or conspiratorial involvement with Oswald.

### Source excerpt:
> "In September /October 1963 Lee Harvey Oswald approached the Soviet Embassy in Mexico City in an attempt to get a visa allowing him to return to the USSR. Kostikov, as a consular officer, handled this visa request. We have no information which indicates any relationship between these individuals other than for the purpose of Oswald's making his visa request."  
— [JFK Files: 104-10012-10022.pdf (page 2)](https://www.archives.gov/files/research/jfk/releases/2025/0318/104-10012-10022.pdf)

Would you like a deeper summary on Kostikov's intelligence background or his later activities?

Great! Our retrieval pipeline is clearly returning highly relevant information to our agent - allowing us to explore the JFK files, as follow up questions, and try to understand the various connections and characters that appear throughout.

Once we're done asking questions, we should ideally delete our vector index to save resources.

In [29]:
pc.delete_index(index_name)

---